In [2]:
import csv
import base64
import time
import fnmatch
import urllib
import traceback
from datetime import datetime, timedelta
from github import Github, StatsContributor
from awesomecure import awesome2py
from urllib.parse import urlparse
from os import getenv, path
from dotenv import load_dotenv
from termcolor import colored

import requests
from bs4 import BeautifulSoup

In [3]:
def crawl_dependents(repo,page_num):
    url = 'https://github.com/{}/network/dependents'.format(repo)
    dependents_data = []
    list_end = False
    
    for i in range(page_num):
        #print("GET " + url)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")

        page_data = [
            "{}/{}".format(
                t.find('a', {"data-repository-hovercards-enabled":""}).text,
                t.find('a', {"data-hovercard-type":"repository"}).text
            )
            for t in soup.findAll("div", {"class": "Box-row"})
        ]
        
        for dependent in page_data:
            if dependent in dependents_data:
                list_end = True 
        
        if list_end:
            break
        else:    
            dependents_data = dependents_data + page_data
        
        try:
            paginationContainer = soup.find("div", {"class":"paginate-container"}).find_all('a')
        except:
            break
        
        try:
            if len(paginationContainer) > 1:
                paginationContainer = paginationContainer[1]
            else:
                paginationContainer = paginationContainer[0]
        except:
            break
        
        if paginationContainer:
            url = paginationContainer["href"]
        else:
            break
        
    return dependents_data

In [4]:
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = "{:02d}:{:02d}".format(mins, secs)
        print(timeformat, end="\r")
        time.sleep(1)
        t -= 1
    print("\n\n\n\n\n")

In [5]:
assert load_dotenv(), 'Environment variables could not be loaded'
g = Github(getenv("GITHUB"))

In [6]:
awesome_url = "https://github.com/protontypes/open-sustainable-technology"
awesome_path = urlparse(awesome_url).path.strip("/")
filename = "README.md"

awesome_repo = g.get_repo(awesome_path)
awesome_content_encoded = awesome_repo.get_contents(
    urllib.parse.quote(filename)
).content
awesome_content = base64.b64decode(awesome_content_encoded)

filehandle = open(".awesome.md", "w")
filehandle.write(awesome_content.decode("utf-8"))
filehandle.close()
repo_dict = awesome2py.AwesomeList(".awesome.md")

print(repo_dict)

Photovoltaics and Solar Energy
 - pvlib-python - A set of documented functions for simulating the performance of photovoltaic energy systems. [https://github.com/pvlib/pvlib-python] - pvfactors - Open source view-factor model for diffuse shading and bifacial PV modeling. [https://github.com/SunPower/pvfactors] - gsee - Global Solar Energy Estimator. [https://github.com/renewables-ninja/gsee] - PVMismatch - An explicit Python PV system IV & PV curve trace calculator which can also calculate mismatch. [https://github.com/SunPower/PVMismatch] - rdtools - An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems. [https://github.com/NREL/rdtools] - Machine-Learning-for-Solar-Energy-Prediction - Predict the power production of a solar panel farm from weather measurements using machine learning. [https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction] - elpv-dataset - A dataset of functional and defective solar c

 - Datasets APIs and open source projects related to Climate Change - A curated list of APIs, open data and ML/AI projects on climate change. [https://github.com/KKulma/climate-change-data] - Awesome Green Software - Research, tools, code, libraries and training to for building applications that emit less carbon into our atmosphere. [https://github.com/Green-Software-Foundation/awesome-green-software] - Awesome Sustainability Jobs - A curated list of companies in the sustainability sector that have jobs for devs. [https://github.com/pogopaule/awesome-sustainability-jobs] - Awesome Spectral Indices - A ready-to-use curated list of Spectral Indices for Remote Sensing applications. [https://github.com/davemlz/awesome-ee-spectral-indices] - Awesome Vegetation Index - List of reference, applications of common Vegetation Indices for Multi-spectral, hyper-spectral and UAV images. [https://github.com/px39n/Awesome-Vegetation-Index] - awesome-transit - Community list of transit APIs, apps, data

In [7]:
csv_projects = open("./csv/projects.csv", "w", newline="")
csv_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "platform",
    "topics",
    "rubric",
    "last_update",
    "stargazers_count",
    "number_of_dependents",
    "stars_last_year",
    "project_active_checkbox",
    "dominating_language",
    "organization",
    "organization_user_name",
    "languages",
    "homepage",
    "created",
    "license",
    "total_commits_last_year",
    "total_number_of_commits",
    "last_issue_closed",
    "open_issues",
    "closed_pullrequests",
    "closed_issues",
    "open_pullrequests",
    "reviews_per_pr",
    "community_development_distribution_score",
    "last_released_date",
    "last_release_tag_name",
    "good_first_issue",
    "contributors",
    "accepts_donations",
    "donation_platforms",
    "code_of_conduct",
    "contribution_guide",
    "organization_avatar",
    "dependents_repos"

]
writer_projects = csv.DictWriter(
    csv_projects, fieldnames=csv_fieldnames
)
writer_projects.writeheader()


csv_github_organizations_fieldnames = [
    "organization_name",
    "github_user_name",
    "github_url",
    "website",
    "location_city",
    "location_country"
    "avatar",
    "public_repos",
    "creation_date",
    "last_update",
    "form_of_organization"
]


csv_github_organizations = open("./csv/github_organizations.csv", "r", newline="")


github_organization_list=[]                            
reader_github_organizations = csv.DictReader(csv_github_organizations)
for entry in reader_github_organizations:
    github_organization_list.append(entry['github_user_name'])
    
csv_github_organizations = open("./csv/github_organizations.csv", "a", newline="")
writer_github_organizations = csv.DictWriter(csv_github_organizations,csv_github_organizations_fieldnames)

In [ ]:
for r in repo_dict.rubrics:
    for entry in r.entries:
        print("------------------------")
        print("Processing %s" % entry.name)
        print("URL: %s" % entry.url)
        if urlparse(entry.url).netloc == "github.com":
            print("%s is a GitHub project" % entry.name)
            try:

                remaining, limit = g.rate_limiting
                resettime = g.rate_limiting_resettime
                min_requests = 300
                if remaining < min_requests:
                    print("------------------------")
                    print("Waiting for more available GitHub requests:")
                    current_time = datetime.now().timestamp()
                    countdown((int(resettime) - int(current_time) + 2))

                print(
                    "GitHub Requests | Limit: {}, Remaining: {}".format(
                        limit, remaining
                    )
                )
                # Gather project information from GitHub
                # https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html
                repo_path = urlparse(entry.url).path.strip("/")
                platform = "github"
                user, project_name = path.split(repo_path)
                repo = g.get_repo(repo_path)
                contents_root = repo.get_contents("")
                releases = repo.get_releases()
                commits = repo.get_commits()
                stargazers = repo.get_stargazers_with_dates()
                
                # Crawel dependents
                try:
                    dependents_repos = crawl_dependents(repo_path,20)
                    
                except Exception as e:
                    print("Dependents not available:")
                    print(e)
                    dependents_repo = []
                    
                    
                number_of_dependents = len(dependents_repos)
                dependents_repos = ",".join(dependents_repos)
                
                # Check if the project is still active
                closed_issues = repo.get_issues(state="closed")
                open_issues = repo.get_issues(state="open")

                closed_pullrequests = repo.get_pulls(state="closed")
                open_pullrequests = repo.get_pulls(state="open")
                
                closed_prs = closed_pullrequests.totalCount
                open_prs = open_pullrequests.totalCount
                
                if closed_prs > 10:
                    pr_review_analyse = 10
                else:
                    pr_review_analyse = closed_prs
                
                total_reviews = 0
                for pull_request in closed_pullrequests[0:pr_review_analyse-1]:
                    pr_reviews = pull_request.get_reviews()
                    total_reviews = total_reviews + pr_reviews.totalCount
                
                try:
                    reviews_per_pr = total_reviews/pr_review_analyse
                    
                except:                   
                    reviews_per_pr = 0

                if closed_issues.totalCount > 0:
                    last_issue_closed = closed_issues[0].updated_at

                total_commits_last_year = 0
                activity = repo.get_stats_commit_activity()
                for event in activity:
                    total_commits_last_year = total_commits_last_year + event.total

                last_issue_closed_timedelta = datetime.utcnow() - last_issue_closed

                if (
                    last_issue_closed_timedelta > timedelta(days=365)
                    and total_commits_last_year == 0
                    or repo.archived
                ):
                    print("%s is an inactive project" % entry.name)
                    project_active_checkbox = "false"
                else:
                    print("%s is an active project" % entry.name)
                    project_active_checkbox = "true"

                try:
                    license = repo.get_license()
                    if license.license.spdx_id == "NOASSERTION":
                        print("Custom license found")
                        license_name = "CUSTOM"
                    else:
                        license_name = license.license.spdx_id
                except:
                    print("No license information found")
                    license_name == "UNDEFINED"

                labels = ",".join([entry.name for entry in repo.get_labels()])
                topics = ",".join(repo.get_topics())

                languages_states = repo.get_languages()
                programming_languages = ",".join(languages_states.keys())
                
                try:
                    dominating_language = list(languages_states.keys())[0]
                except:
                    dominating_language = ""

                try:
                    dotfolder_file = repo.get_contents(".github")
                    for file in dotfolder_file:
                        if file.path == ".github/FUNDING.yml":
                            print("Funding found")
                            donation_platforms = base64.b64decode(file.content)
                            accepts_donations = "true"
                        else:
                            donation_platforms = None
                            accepts_donations = "false"
                except:
                    print("No funding information found")
                    donation_platforms = None
                    accepts_donations = "false"

                try:
                    code_of_conduct = "false"
                    contribution_guide = "false"
                    for file_content in contents_root:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"

                    for file_content in dotfolder_file:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"

                except Exception as e:
                    print(e)

                contributors = repo.get_stats_contributors()
                contributor_activity = {}
                commits_total = 0
                for individuum in contributors:
                    contributor_activity[individuum.author.login] = individuum.total
                    commits_total = commits_total + individuum.total

                sorted_contributor = dict(
                    sorted(contributor_activity.items(), key=lambda item: item[1])
                )
                weighted_contribution = {
                    k: v / commits_total for k, v in sorted_contributor.items()
                }

                # Create a very simple community health score that shows how much the project is focused on one developer
                community_development_distribution_score = 1 - max(
                    weighted_contribution.values()
                )

                try:
                    last_released_date = releases[0].published_at.strftime(
                        "%Y/%m/%d, %H:%M:%S"
                    )
                    last_release_tag_name = releases[0].tag_name

                except Exception as e:
                    print("No Release found")
                    last_released_date = ""
                    last_release_tag_name = ""
                    print(e)

                total_number_of_commits = commits.totalCount
                
                stars_last_year = 0
                for star in stargazers:
                    starred_delta = datetime.utcnow() - star.starred_at
                    if  starred_delta < timedelta(days=365):
                        stars_last_year = stars_last_year + 1

                # Gathering organization data
                if repo.organization is None:
                    print("No Organization found.Project in user namespace.")
                    organization_user_name = None

                elif repo.organization.login not in github_organization_list:
                    print("Organization not in list. Gathering data.")
                    organization_user_name = repo.organization.login
                    organization_name = repo.organization.name
                    organization_avatar = repo.organization.avatar_url
                    organization_location = repo.organization.location
                    organization_github_url = repo.organization.html_url
                    organization_website = repo.organization.blog
                    organization_created = repo.organization.created_at.strftime("%Y/%m/%d, %H:%M:%S")
                        
                        
                    organization_data = {
                        "organization_name": organization_name,
                        "github_user_name":organization_user_name,
                        "organization_github_url":organization_github_url,
                        "website":organization_website,
                        "avatar": organization_avatar,
                        "location_city": organization_location,
                        "creation_date": organization_created,
                    }

                    github_organization_list.append(organization_user_name)
                    print("Organization Data:")
                    print(organization_data)
                    writer_github_organizations.writerow(organization_data)

                else:
                    organization_name = repo.organization.login
                    organization_user_name = repo.organization.name
                    organization_avatar = repo.organization.avatar_url
                    

                project_data = {
                    "project_name": entry.name,
                    "git_namespace": user,
                    "git_url": repo.clone_url,
                    "rubric": r.key,
                    "oneliner": entry.text[2:],
                    "topics": topics,
                    "organization": organization_name,
                    "organization_user_name": organization_user_name,
                    "created": repo.created_at.strftime("%Y/%m/%d, %H:%M:%S"),
                    "last_update": repo.updated_at.strftime("%Y/%m/%d, %H:%M:%S"),
                    "total_commits_last_year": total_commits_last_year,
                    "project_active_checkbox": project_active_checkbox,
                    "last_issue_closed": last_issue_closed.strftime(
                        "%Y/%m/%d, %H:%M:%S"
                    ),
                    "last_released_date": last_released_date,
                    "last_release_tag_name": last_release_tag_name,
                    "total_number_of_commits": total_number_of_commits,
                    "community_development_distribution_score": community_development_distribution_score,
                    "stargazers_count": repo.stargazers_count,
                    "number_of_dependents": number_of_dependents,
                    "dependents_repos": dependents_repos,
                    "stars_last_year": stars_last_year,
                    "dominating_language": dominating_language,
                    "languages": programming_languages,
                    "homepage": repo.homepage,
                    "closed_issues": closed_issues.totalCount,
                    "open_issues": open_issues.totalCount,
                    "closed_pullrequests": closed_prs,
                    "open_pullrequests": open_prs,
                    "reviews_per_pr": reviews_per_pr,
                    "good_first_issue": repo.get_issues(
                        state="open", labels=["good first issue"]
                    ).totalCount,
                    "license": license_name,
                    "contributors": repo.get_contributors().totalCount,
                    "accepts_donations": accepts_donations,
                    "donation_platforms": donation_platforms,
                    "code_of_conduct": code_of_conduct,
                    "contribution_guide": contribution_guide,
                    "organization_avatar":organization_avatar,
                    "platform":platform
                }

                print("Project Data:")
                print(project_data)
                writer_projects.writerow(project_data)

            except Exception as e:
                print(colored("Failed to gather project information:"))
                print(colored(e, "red"))
                print(traceback.format_exc())

        elif urlparse(entry.url).netloc == "gitlab.com":
            print("%s is a Gitlab project" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)
            platform = "gitlab"

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "git_url": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
                "platform":platform
            }
            print("Project Data:")
            print(project_data)
            writer_projects.writerow(project_data)

        else:
            print("%s is hosted on custom platform" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)
            platform = "custom"

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "homepage": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
                "platform":platform
            }

            print("Project Data:")
            print(project_data)
            writer_projects.writerow(project_data)

writer_projects.close()
writer_github_organizations.close()

------------------------
Processing pvlib-python
URL: https://github.com/pvlib/pvlib-python
pvlib-python is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 4998
pvlib-python is an active project
Code of conduct found
Contribution guide found
Project Data:
{'project_name': 'pvlib-python', 'git_namespace': 'pvlib', 'git_url': 'https://github.com/pvlib/pvlib-python.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A set of documented functions for simulating the performance of photovoltaic energy systems.', 'topics': 'solar-energy,python,renewable-energy,renewables,photovoltaic', 'organization': 'pvlib', 'organization_user_name': 'pvlib', 'created': '2015/02/17, 00:21:33', 'last_update': '2022/02/16, 08:39:42', 'total_commits_last_year': 88, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/02/18, 04:06:44', 'last_released_date': '2021/09/01, 22:49:24', 'last_release_tag_name': 'v0.9.0', 'total_number_of_commits': 1379, 'community_development_distributi

{'project_name': 'pvfactors', 'git_namespace': 'SunPower', 'git_url': 'https://github.com/SunPower/pvfactors.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Open source view-factor model for diffuse shading and bifacial PV modeling.', 'topics': 'solar-energy,renewable-energy,python,bifacial', 'organization': 'SunPower', 'organization_user_name': 'SunPower', 'created': '2018/05/14, 06:10:55', 'last_update': '2022/01/13, 00:44:36', 'total_commits_last_year': 8, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/01/20, 23:49:19', 'last_released_date': '2021/03/27, 13:39:44', 'last_release_tag_name': 'v1.5.1', 'total_number_of_commits': 102, 'community_development_distribution_score': 0.10989010989010994, 'stargazers_count': 55, 'number_of_dependents': 5, 'dependents_repos': 'thesethtruth/LESO,prathamrg/solarapp,anomam/pvfactors_iea_pvps_study,toddkarin/vocmax,toddkarin/pvtools', 'stars_last_year': 13, 'dominating_language': 'Python', 'languages': 'Python,Makefile

{'project_name': 'Machine-Learning-for-Solar-Energy-Prediction', 'git_namespace': 'ColasGael', 'git_url': 'https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Predict the power production of a solar panel farm from weather measurements using machine learning.', 'topics': 'machine-learning,neural-network,data-processing,python,matlab,tensorflow', 'organization': 'NREL', 'organization_user_name': None, 'created': '2018/05/06, 19:43:04', 'last_update': '2022/02/18, 08:34:47', 'total_commits_last_year': 0, 'project_active_checkbox': 'false', 'last_issue_closed': '2019/11/15, 19:01:02', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 11, 'community_development_distribution_score': 0.0, 'stargazers_count': 122, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 36, 'dominating_language': 'Python', 'languages': 'Python,MATLAB,R,TeX,TSQL', 'homepage': '', 'c

No Organization found.Project in user namespace.
Project Data:
{'project_name': 'pvtrace', 'git_namespace': 'danieljfarrell', 'git_url': 'https://github.com/danieljfarrell/pvtrace.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Optical ray tracing for luminescent materials and spectral converter photovoltaic devices.', 'topics': 'python,photovoltaics,raytracing,optics,energy', 'organization': 'pvcaptest', 'organization_user_name': None, 'created': '2011/01/05, 14:28:41', 'last_update': '2021/12/21, 09:13:10', 'total_commits_last_year': 1, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/02/18, 13:43:32', 'last_released_date': '2020/11/18, 23:04:23', 'last_release_tag_name': '2.1.6', 'total_number_of_commits': 185, 'community_development_distribution_score': 0.019108280254777066, 'stargazers_count': 79, 'number_of_dependents': 1, 'dependents_repos': 'danieljfarrell/pvtrace', 'stars_last_year': 9, 'dominating_language': 'Python', 'languages': 'Python', 'homepa

{'project_name': 'bifacial_radiance', 'git_namespace': 'NREL', 'git_url': 'https://github.com/NREL/bifacial_radiance.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Toolkit for working with RADIANCE for the ray-trace modeling of Bifacial Photovoltaics.', 'topics': 'radiance,bifacial,photovoltaics,renewable-energy,renewables,gui', 'organization': 'NREL', 'organization_user_name': 'National Renewable Energy Laboratory', 'created': '2017/12/13, 17:56:18', 'last_update': '2022/02/05, 21:46:04', 'total_commits_last_year': 319, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/02/18, 16:46:05', 'last_released_date': '2022/02/14, 21:04:38', 'last_release_tag_name': 'v0.4.0-dev2', 'total_number_of_commits': 1429, 'community_development_distribution_score': 0.36534839924670437, 'stargazers_count': 53, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 14, 'dominating_language': 'Python', 'languages': 'Python,C++,TeX,C,Shell,Makefile,Awk', 'homepage'

...

{'project_name': 'antaresRead', 'git_namespace': 'rte-antares-rpackage', 'git_url': 'https://github.com/rte-antares-rpackage/antaresRead.git', 'rubric': 'Energy Modeling and Optimization', 'oneliner': 'Import, manipulate and explore the results of an Antares simulation.', 'topics': 'r,monte-carlo-simulation,optimisation,simulation,linear-algebra,energy,electricity,adequacy,rhdf5,hdf5,rte,tyndp,bilan,previsionnel', 'organization': 'rte-antares-rpackage', 'organization_user_name': 'rte-antares-rpackage', 'created': '2016/11/15, 15:26:32', 'last_update': '2021/12/22, 14:08:38', 'total_commits_last_year': 74, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/01/13, 09:21:36', 'last_released_date': '2021/11/16, 13:50:33', 'last_release_tag_name': 'v2.2.9', 'total_number_of_commits': 912, 'community_development_distribution_score': 0.5652173913043479, 'stargazers_count': 10, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 1, 'dominating_language': 'R', 'lang

{'project_name': 'energy-py-linear', 'git_namespace': 'ADGEfficiency', 'git_url': 'https://github.com/ADGEfficiency/energy-py-linear.git', 'rubric': 'Energy Modeling and Optimization', 'oneliner': 'Optimizing energy systems using mixed integer linear programming.', 'topics': 'python,optimization,energy', 'organization': 'reegis', 'organization_user_name': None, 'created': '2019/01/15, 21:59:41', 'last_update': '2022/02/17, 16:10:42', 'total_commits_last_year': 21, 'project_active_checkbox': 'true', 'last_issue_closed': '2020/11/23, 02:42:31', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 114, 'community_development_distribution_score': 0.26262626262626265, 'stargazers_count': 28, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 6, 'dominating_language': 'Jupyter Notebook', 'languages': 'Jupyter Notebook,Python,Makefile', 'homepage': '', 'closed_issues': 16, 'open_issues': 1, 'closed_pullrequests': 8, 'open_pullrequests': 0, 'revi

CapacityExpansion.jl is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 553
CapacityExpansion.jl is an active project
No funding information found
Contribution guide found
No Organization found.Project in user namespace.
Project Data:
{'project_name': 'CapacityExpansion.jl', 'git_namespace': 'YoungFaithful', 'git_url': 'https://github.com/YoungFaithful/CapacityExpansion.jl.git', 'rubric': 'Energy Modeling and Optimization', 'oneliner': 'Future energy system planning (Generation and Transmission Capacity Expansion Planning) in Julia.', 'topics': 'capacity-expansion-planning,energy,energy-optimization-model,julia,jump,clustering,california,germany', 'organization': 'NREL', 'organization_user_name': None, 'created': '2019/03/28, 21:02:36', 'last_update': '2021/12/28, 21:47:17', 'total_commits_last_year': 1, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/04/27, 07:58:56', 'last_released_date': '2020/09/30, 18:35:18', 'last_release_tag_name': 'v0.2.2', 'total_numbe

{'project_name': 'Open Energy Platform', 'git_namespace': 'OpenEnergyPlatform', 'git_url': 'https://github.com/OpenEnergyPlatform/oeplatform.git', 'rubric': 'Energy Modeling and Optimization', 'oneliner': 'Aims to ensure quality, transparency and reproducibility in energy system research. It is a collection of various tools and information and that help working with energy-related data.', 'topics': 'oep,django,rest-api,energy,database,open-energy-family,open-data', 'organization': 'OpenEnergyPlatform', 'organization_user_name': 'Open Energy Family', 'created': '2015/11/20, 14:21:02', 'last_update': '2022/01/18, 22:15:05', 'total_commits_last_year': 181, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/01/27, 12:33:51', 'last_released_date': '2021/09/27, 07:06:43', 'last_release_tag_name': 'v0.9.4', 'total_number_of_commits': 2717, 'community_development_distribution_score': 0.30082987551867224, 'stargazers_count': 46, 'number_of_dependents': 0, 'dependents_repos': '', 'sta

{'project_name': 'pyGRETA', 'git_namespace': 'tum-ens', 'git_url': 'https://github.com/tum-ens/pyGRETA.git', 'rubric': 'Energy Modeling and Optimization', 'oneliner': 'Python Generator of REnewable Time series and mAps: a tool that generates high-resolution potential maps and time series for user-defined regions within the globe.', 'topics': 'renewable-energy,renewable-timeseries,potentials,wind,pv,csp,gis,high-resolution', 'organization': 'tum-ens', 'organization_user_name': 'Chair of Renewable and Sustainable Energy Systems', 'created': '2019/03/08, 17:08:33', 'last_update': '2022/02/07, 13:35:56', 'total_commits_last_year': 120, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/10/11, 10:38:26', 'last_released_date': '2021/09/29, 08:52:40', 'last_release_tag_name': 'v2.0.0', 'total_number_of_commits': 848, 'community_development_distribution_score': 0.5158730158730158, 'stargazers_count': 24, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 5, 'domin